In [1]:
import pandas as pd

## Explorations datasets

### I- Data: traindemographics dataset

In [2]:
train_dg = pd.read_csv("../data/raw_data/train/traindemographics.csv")
train_dg.head()

,customerid,birthdate,bank_account_type,longitude_gps,latitude_gps,bank_name_clients,bank_branch_clients,employment_status_clients,level_of_education_clients
0,8a858e135cb22031015cbafc76964ebd,1973-10-10 00:00:00.000000,Savings,3.319219,6.528604,GT Bank,NaN,NaN,NaN
1,8a858e275c7ea5ec015c82482d7c3996,1986-01-21 00:00:00.000000,Savings,3.325598,7.119403,Sterling Bank,NaN,Permanent,NaN
2,8a858e5b5bd99460015bdc95cd485634,1987-04-01 00:00:00.000000,Savings,5.746100,5.563174,Fidelity Bank,NaN,NaN,NaN
3,8a858efd5ca70688015cabd1f1e94b55,1991-07-19 00:00:00.000000,Savings,3.362850,6.642485,GT Bank,NaN,Permanent,NaN
4,8a858e785acd3412015acd48f4920d04,1982-11-22 00:00:00.000000,Savings,8.455332,11.971410,GT Bank,NaN,Permanent,NaN


In [4]:
train_dg.shape

(4346, 9)

In [7]:
# Purcentage of data that is missing
train_dg.isnull().sum()/train_dg.shape[0] * 100

customerid                     0.000000
birthdate                      0.000000
bank_account_type              0.000000
longitude_gps                  0.000000
latitude_gps                   0.000000
bank_name_clients              0.000000
bank_branch_clients           98.826507
employment_status_clients     14.910262
level_of_education_clients    86.493327
dtype: float64

We notice that the column bank_branch_clients and level_of_education_clients and mainly missing more than 85%. We will remove those two columns. We will impute the missing values of the column employment_status_clients